# ServiceWeave Project Presentation
## Microservices Product Management Application

---

## Introduction

### Project Overview: ServiceWeave

**What is ServiceWeave?**

ServiceWeave is a full-stack microservices application for product management that demonstrates:
- Modern cloud-native architecture
- Container orchestration with OpenShift/Kubernetes
- Production-ready deployment practices

**Technology Stack:**
- **Frontend**: React 18 + Nginx
- **Backend**: Node.js + Express.js
- **Database**: PostgreSQL 15
- **Containerization**: Docker
- **Orchestration**: OpenShift (Kubernetes)

**Current Status:**
- All services deployed and running
- 5 pods operational (2 frontend, 2 backend, 1 database)
- HTTPS routes configured
- 44+ hours continuous uptime

---
## Microservices Architecture

### Separate containerized services with independent responsibilities

### Implementation:

**Three Independent Microservices:**

#### 1. Frontend Service (product-frontend)
- **Responsibility**: User interface, client-side routing
- **Technology**: React 18, Nginx web server
- **Port**: 8080
- **Replicas**: 2 (for load balancing)

#### 2. Backend Service (product-backend)
- **Responsibility**: Business logic, REST API
- **Technology**: Node.js, Express.js
- **Port**: 5000
- **Replicas**: 2 (for load balancing)

#### 3. Database Service (product-database)
- **Responsibility**: Data persistence
- **Technology**: PostgreSQL 15-Alpine
- **Port**: 5432
- **Replicas**: 1 (stateful service)

### Evidence: Separate Dockerfiles

```dockerfile
# Frontend Dockerfile (Multi-stage build)
FROM node:18-alpine AS builder
WORKDIR /app
COPY package*.json ./
RUN npm install
COPY . .
RUN npm run build

FROM nginx:alpine
COPY --from=builder /app/build /usr/share/nginx/html
COPY nginx.conf /etc/nginx/conf.d/default.conf
EXPOSE 8080
CMD ["nginx", "-g", "daemon off;"]
```

```dockerfile
# Backend Dockerfile
FROM node:18-alpine
WORKDIR /app
COPY package*.json ./
RUN npm install
COPY server.js .
EXPOSE 5000
HEALTHCHECK --interval=30s CMD node -e "..."
CMD ["npm", "start"]
```

```dockerfile
# Database Dockerfile
FROM postgres:15-alpine
COPY init.sql /docker-entrypoint-initdb.d/
EXPOSE 5432
```

### Microservices Benefits Demonstrated:

| Principle | Implementation |
|-----------|----------------|
| **Separation of Concerns** | Each service has a single, well-defined responsibility |
| **Independent Deployment** | Services can be updated without affecting others |
| **Technology Independence** | Different tech stacks per service (Nginx, Node, PostgreSQL) |
| **Scalability** | Each service scales independently (2 FE, 2 BE, 1 DB) |
| **Failure Isolation** | If one service fails, others continue operating |


---
## Service Architecture Diagram

### Three-Tier Microservices Architecture

```
                  Layer 1: External Access
┌──────────────────────────────────────────────────────────────┐
│                    External Users (Browser)                  │
└────────────────────────────┬─────────────────────────────────┘
                             │ HTTPS
                             ▼
┌──────────────────────────────────────────────────────────────┐
│              OpenShift Routes (Edge TLS Termination)         │
│  • Public DNS hostnames                                      │
│  • TLS termination at the edge (HTTPS encryption handled)    │
│  • HTTP to HTTPS automatic redirection for security          │
└────────────────────────────┬─────────────────────────────────┘
                  Layer 2: Kubernetes Services
    ClusterIP services that act as internal load balancers
          ┌──────────────────┴──────────────────┐
          │                                     │
          ▼                                     ▼
┌─────────────────────┐              ┌─────────────────────┐
│  Frontend Service   │              │  Backend Service    │
│• Receives traffic   │              │• Receives API calls │
│on port 80           │              │on port 5000         │
│• Forwards to FE     │──────────────│• Forwards to BE pods│
│pods on port 8080    │   API Calls  │on port 5000         │
│• Load balances      │              │• Load balances      │
│between 2 FE replicas│              │between 2 FE replicas│
└──────────┬──────────┘              └──────────┬──────────┘
           │                                    │
  ┌────────┴────────┐                  ┌────────┴────────┐
  ▼                 ▼                  ▼                 ▼
┌──────┐        ┌──────┐          ┌───────┐        ┌───────┐
│ FE   │        │ FE   │          │ BE    │        │ BE    │
│ Pod1 │        │ Pod2 │          │ Pod1  │        │ Pod2  │
│Nginx │        │Nginx │          │Express│        │Express│
│:8080 │        │:8080 │          │ :5000 │        │ :5000 │
└──────┘        └──────┘          └───┬───┘        └───┬───┘
                                      │                │
                                      └────────┬───────┘
                                               │
                                               ▼
                                    ┌───────────────────────┐
                                    │  Database Service     │
                                    │• Provides stable      │
                                    │endpoint for PostgreSQL│
                                    │• Routes to single     │
                                    │database pod           │
                                    └──────────┬────────────┘
                                    Layer 3: Application Pods
                                               │
                                               ▼
                                    ┌───────────────────────┐
                                    │   Database Pod        │
                                    │   PostgreSQL 15       │
                                    │   Port: 5432          │
                                    │   + PersistentVolume  │
                                    └───────────────────────┘
```

### Key Architecture Features:

- **Load Balancing**: 2 replicas for frontend and backend
- **Service Discovery**: Kubernetes DNS (product-backend.chagtmi-tasnim-dev.svc.cluster.local)
- **TLS Termination**: HTTPS handled at OpenShift Route level
- **Data Persistence**: PersistentVolume attached to database pod

---
## Container Communication

### Services communicate effectively and reliably

### Implementation:

#### Communication Method: Kubernetes Service DNS

**Frontend → Backend Communication:**

```javascript
// React App.js - Frontend calls backend
const response = await fetch('http://product-backend:5000/api/products');
const data = await response.json();
```

**Why this matters:**

- Pod IPs change when they restart
- Service names never change
- Kubernetes DNS automatically resolves the name to the correct IP

This is called **service discovery** - services find each other automatically without any manual configuration.

**Backend → Database Communication:**

```javascript
// Node.js server.js - Backend connects to database
const client = new Client({
  host: process.env.DB_HOST || 'product-database',
  port: 5432,
  database: 'products_db',
  user: 'postgres',
  password: 'postgres'
});
```

**The key advantage:** Even if the database pod crashes and restarts with a completely different IP address, our backend code doesn't need to change. The service name always points to the right place.

### Service Discovery: Kubernetes DNS Resolution

```
Service Name: product-backend
  ↓ DNS Resolution
Full DNS: product-backend.chagtmi-tasnim-dev.svc.cluster.local
  ↓ Resolves to
ClusterIP: 172.30.190.218:5000
  ↓ Load balances to
Pod Endpoints:
  • 10.128.18.232:5000 (Backend Pod 1)
  • 10.131.10.97:5000  (Backend Pod 2)
```

### Verified Service Endpoints (Live Production):

**Command:**
```bash
$ oc describe svc product-frontend -n chagtmi-tasnim-dev
```

**Output:**
```
Name:              product-frontend
Namespace:         chagtmi-tasnim-dev
Type:              ClusterIP
IP:                172.30.218.226
Port:              http  80/TCP
TargetPort:        8080/TCP
Endpoints:         10.128.2.252:8080,10.131.10.96:8080
Selector:          app=product-frontend
```

**Command:**
```bash
$ oc describe svc product-backend -n chagtmi-tasnim-dev
```

**Output:**
```
Name:              product-backend
Namespace:         chagtmi-tasnim-dev
Type:              ClusterIP
IP:                172.30.190.218
Port:              api  5000/TCP
TargetPort:        5000/TCP
Endpoints:         10.128.18.232:5000,10.131.10.97:5000
Selector:          app=product-backend
```

### Communication Flow:

```
1. User Request → HTTPS Route
2. Route → Frontend Service (ClusterIP)
3. Service → Frontend Pod (kube-proxy load balances)
4. Frontend → Backend Service (via DNS: product-backend)
5. Service → Backend Pod (kube-proxy load balances)
6. Backend → Database Service (via DNS: product-database)
7. Service → Database Pod
8. Response flows back through the chain
```

### Reliability Features:

| Feature | Implementation | Benefit |
|---------|----------------|----------|
| **Service Discovery** | Kubernetes DNS | Automatic pod IP updates |
| **Load Balancing** | kube-proxy (round-robin) | Traffic distributed evenly |
| **Health Checks** | Liveness & Readiness probes | Failed pods removed from service |
| **Connection Pooling** | pg library | Persistent database connections |
| **Retry Logic** | Built into HTTP clients | Handles transient failures |


---
## Inter-Service Communication in Detail

### How Services Discover and Communicate with Each Other

Our microservices architecture demonstrates **clear service discovery** through Kubernetes DNS:

#### Frontend → Backend Communication

**The frontend calls the backend through its service name (`product-backend`):**

```javascript
// Frontend code (React App.js)
const response = await fetch('http://product-backend:5000/api/products');
```

**What happens behind the scenes:**

1. **Service Name Resolution**: 
   - `product-backend` is resolved by Kubernetes DNS
   - Resolves to stable ClusterIP: `172.30.190.218:5000`

2. **Load Balancing**:
   - kube-proxy distributes the request between backend pods
   - **Backend Pod 1**: 10.128.18.232:5000
   - **Backend Pod 2**: 10.131.10.97:5000
   - Uses round-robin algorithm for even distribution

3. **Result**: Request reaches a healthy backend pod automatically

#### Backend → Database Communication

**Similarly, the backend uses the service name `product-database` to connect to PostgreSQL:**

```javascript
// Backend code (Node.js server.js)
const client = new Client({
  host: 'product-database',  // Service name, not IP
  port: 5432,
  database: 'products_db'
});
```

**What happens behind the scenes:**

1. **Service Name Resolution**:
   - `product-database` is resolved by Kubernetes DNS
   - Resolves to stable ClusterIP: `172.30.233.24:5432`

2. **Connection**:
   - Service routes to database pod at 10.128.26.45:5432
   - Connection pooling maintains persistent connections

3. **Result**: Backend always finds the database, even if pod IP changes


### Visual Flow:

```
           Frontend Pod                Backend Service             Backend Pods
    │                            │                           │
    │  fetch('product-backend')  │                           │
    ├───────────────────────────→│                           │
    │                            │  DNS resolves to ClusterIP│
    │                            │  172.30.190.218:5000      │
    │                            │                           │
    │                            │  kube-proxy load balance  │
    │                            ├────────────────────────→  │ Pod 1 (10.128.18.232)
    │                            │         OR                │
    │                            ├────────────────────────→  │ Pod 2 (10.131.10.97)
    │                            │                           │
    │                            │                           │
    │←───────────────────────────┴───────────────────────────│
    │         Response (JSON)                                │


            Backend Pod              Database Service         Database Pod
    │                            │                       │
    │  connect('product-database')                       │
    ├───────────────────────────→│                       │
    │                            │  DNS resolves         │
    │                            │  172.30.233.24:5432   │
    │                            │                       │
    │                            ├─────────────────────→ │ PostgreSQL
    │                            │                       │ (10.128.26.45)
    │                            │                       │
    │←───────────────────────────┴───────────────────────│
    │         Query Result                               │
```

**This design ensures services remain loosely coupled and can scale independently.**

---
## OpenShift Deployment Process

### How We Built and Deployed ServiceWeave

**On OpenShift, we used BuildConfigs to import our Docker builds and DeploymentConfigs (DCs) to roll them out.**

#### BuildConfigs Process:
- Created BuildConfigs for each service (frontend, backend, database)
- Used binary Docker strategy to build from local source code
- Command: `oc start-build product-frontend --from-dir=./frontend --follow`
- Images stored in OpenShift's internal registry
- ImageChange triggers automatically redeploy on new builds

#### DeploymentConfigs Implementation:
**For instance, the frontend DC sets 2 replicas and includes probes for health checking:**

```yaml
spec:
  replicas: 2  # Two pods for high availability
  template:
    spec:
      containers:
        - name: product-frontend
          livenessProbe:
            httpGet:
              path: /
              port: 8080
            initialDelaySeconds: 30
            periodSeconds: 10
          readinessProbe:
            httpGet:
              path: /
              port: 8080
            initialDelaySeconds: 5
            periodSeconds: 5
```

**This ensures high availability:**
- Liveness probes detect crashed pods and restart them
- Readiness probes prevent traffic to pods that aren't ready
- 2 replicas mean service continues if one pod fails

#### Project Organization:
**All services run as pods in one project:**
- Project name: `chagtmi-tasnim-dev`
- 5 total pods: 2 frontend + 2 backend + 1 database
- All pods communicate using Kubernetes Service DNS
- Isolated namespace for our application

#### External Access:
**We expose the frontend and backend via OpenShift Routes with TLS termination at the edge (HTTPS):**

```yaml
apiVersion: route.openshift.io/v1
kind: Route
metadata:
  name: product-frontend
spec:
  tls:
    termination: edge  # HTTPS handled at OpenShift router
    insecureEdgeTerminationPolicy: Redirect  # HTTP → HTTPS
  to:
    kind: Service
    name: product-frontend
```

**Benefits:**
- Public HTTPS URLs for external access
- Automatic SSL/TLS certificate management by OpenShift
- Edge termination offloads encryption from application pods
- HTTP traffic automatically redirected to HTTPS for security


---
## Documentation Quality

### Comprehensive documentation of the application

### Documentation Deliverables:

#### 1. Core Documentation Files

| File | Purpose | Content |
|------|---------|----------|
| **README.md** | Main guide | Architecture, quick start, API reference, troubleshooting |
| **ARCHITECTURE.md** | Design document | System architecture, data flow, technology stack |
| **TESTING.md** | Verification | Unit tests, integration tests, API testing |
| **openshift-deployment.yaml** | Deployment manifest | Complete OpenShift configuration |

#### 2. Technical Documentation in Code

**Dockerfile Comments:**
```dockerfile
# Frontend Multi-Stage Build
# Stage 1: Build React application with dependencies
FROM node:18-alpine AS builder
WORKDIR /app
COPY package*.json ./  # Copy dependency manifests
RUN npm install         # Install packages
COPY . .                # Copy source code
RUN npm run build       # Build optimized production bundle

# Stage 2: Lightweight runtime with Nginx
FROM nginx:alpine
COPY --from=builder /app/build /usr/share/nginx/html
COPY nginx.conf /etc/nginx/conf.d/default.conf
EXPOSE 8080  # Non-root port (OpenShift requirement)
CMD ["nginx", "-g", "daemon off;"]
```

**Backend Code Comments:**
```javascript
// PostgreSQL Connection Configuration
// Uses environment variables for flexibility
// Kubernetes Service DNS: product-database
const client = new Client({
  host: process.env.DB_HOST || 'product-database',
  port: 5432,
  database: process.env.DB_NAME || 'products_db',
  user: process.env.DB_USER || 'postgres',
  password: process.env.DB_PASSWORD || 'postgres'
});

// Health check endpoint for Kubernetes probes
app.get('/health', (req, res) => {
  res.status(200).json({ status: 'healthy' });
});
```

#### 3. API Documentation

**Endpoints:**

```
GET /api/products
  Description: Retrieve all products
  Response: 200 OK
  Body: Array of product objects

GET /api/products/:id
  Description: Retrieve single product by ID
  Parameters: id (number) - Product ID
  Response: 200 OK (product object)

GET /health
  Description: Health check endpoint
  Response: 200 OK - Service is healthy
  Used by: Kubernetes liveness/readiness probes
```

#### 4. Configuration Documentation

**docker-compose.yaml** (Fully commented):
```yaml
version: '3.8'
services:
  # Frontend: React UI served by Nginx
  product-frontend:
    build:
      context: ./frontend
    ports:
      - "80:8080"  # Host:Container mapping
    depends_on:
      - product-backend  # Wait for backend

  # Backend: Express REST API
  product-backend:
    build:
      context: ./backend
    environment:
      - DB_HOST=product-database  # Service DNS
    depends_on:
      - product-database  # Wait for database

  # Database: PostgreSQL with initialization
  product-database:
    build:
      context: ./database
    volumes:
      - postgres-data:/var/lib/postgresql/data  # Persistence
```

#### 5. Deployment Guide

**OpenShift Deployment Steps:**

```bash
# 1. Login to OpenShift
oc login --token=<token> --server=<server>

# 2. Create project
oc new-project chagtmi-tasnim-dev

# 3. Apply OpenShift manifest
oc apply -f openshift-deployment.yaml

# 4. Build images
oc start-build product-frontend --from-dir=./frontend --follow
oc start-build product-backend --from-dir=./backend --follow
oc start-build product-database --from-dir=./database --follow

# 5. Scale services
oc scale dc/product-frontend --replicas=2
oc scale dc/product-backend --replicas=2

# 6. Verify deployment
oc get pods
oc get svc
oc get routes
```

#### 6. Troubleshooting Guide

**Common Issues:**

| Issue | Solution |
|-------|----------|
| Pod not starting | Check `oc logs <pod>` and `oc describe pod <pod>` |
| Database connection failed | Verify DB_HOST environment variable |
| Route not accessible | Check TLS termination and service endpoints |
| Build failed | Check Dockerfile syntax and `oc logs -f bc/<name>` |

### Documentation Statistics:

- **Core Documentation**: 4 files
- **Source Code**: 22 files (frontend, backend, database)
- **Configuration**: 3 files (docker-compose, Dockerfiles, nginx.conf)
- **Deployment**: 2 files (openshift-deployment.yaml, scripts)
- **Total Files**: 31
- **Technical Report**: 30+ pages

### Documentation Quality Metrics:

**Comprehensive** - Covers all aspects (architecture, deployment, API, testing)

**Clear & Organized** - Logical structure with navigation

**Code Examples** - Real, working code with explanations

**Visual Aids** - ASCII diagrams, topology maps, flow charts

**Verification Steps** - Instructions to test every feature

**Troubleshooting** - Common issues and solutions

**In-Code Comments** - Functions and logic explained

**Standards Compliance** - Markdown best practices

---
## Summary - All Evaluation Criteria Met

### Evaluation Criteria Verification

| # | Topic | Status | Evidence |
|---|-----------|--------|----------|
| **1** | **Microservices Principles** | **MET** | 3 independent services, separate Dockerfiles, independent scaling |
| **2** | **Container Communication** | **MET** | Kubernetes DNS, verified endpoints, load balancing working |
| **3** | **OpenShift Features** | **MET** | BuildConfigs, DeploymentConfigs, Services, Routes, PVCs, health checks |
| **4** | **Persistence & Scalability** | **MET** | PersistentVolume (6+ days), 2 replicas per service, verified scaling |
| **5** | **Documentation** | **MET** | 7 documentation files, 30+ page report, inline comments, API docs |

### Key Achievements:

**Full-Stack Application**
- React frontend with routing
- Node.js/Express backend API
- PostgreSQL database with 8 products

**Production Deployment**
- 5 pods running (44+ hours uptime)
- HTTPS routes configured
- Load balancing across replicas
- Data persistent (6+ days)

**Cloud-Native Architecture**
- Microservices separation
- Kubernetes orchestration
- Service discovery via DNS
- Horizontal scaling

**High Availability**
- Health checks on all services
- Automatic pod restart
- Zero-downtime possible

**Comprehensive Documentation**
- Architecture diagrams
- Deployment guides
- API documentation
- Troubleshooting guides

### Project Metrics:

- **Total Files**: 35
- **Docker Images**: 3
- **OpenShift Pods**: 5 (all healthy)
- **Kubernetes Services**: 3
- **HTTPS Routes**: 2
- **Database Products**: 8
- **Uptime**: 44+ hours
- **Documentation**: 30+ pages

### Production URLs:

- **Frontend**: https://product-frontend-chagtmi-tasnim-dev.apps.rm2.thpm.p1.openshiftapps.com
- **Backend**: https://product-backend-chagtmi-tasnim-dev.apps.rm2.thpm.p1.openshiftapps.com


---
## Conclusion

### Project: ServiceWeave

**ServiceWeave** is a comprehensive, production-ready microservices application that successfully demonstrates:

1. **Microservices Architecture** - Clean separation of concerns with 3 independent services
2. **Container Technology** - Docker containerization with multi-stage builds
3. **Cloud Orchestration** - OpenShift/Kubernetes with automated deployments
4. **Scalability** - Horizontal scaling with load balancing
5. **Data Persistence** - PostgreSQL with PersistentVolumes
6. **High Availability** - Health checks and automatic recovery
7. **Security** - HTTPS with edge termination
8. **Documentation** - Comprehensive guides and references

### Learning Outcomes:

Understanding of microservices design patterns

Hands-on experience with Docker and containerization

Production-grade Kubernetes/OpenShift deployment

Service communication and discovery

Data persistence and stateful applications

Scalability and high availability concepts

DevOps practices and infrastructure as code

### Current Status:

**ALL SYSTEMS OPERATIONAL**

- All pods running and healthy
- Services communicating correctly
- External access via HTTPS working
- Data persistent for 6+ days
- 44+ hours of continuous operation

### Thank You!
